# Data collecting, cleaning and manipulation

## Step one - loading dataset

In [1]:
# To load dataset, we need to import the libraries Pandas (for data manpulation), NumPy (For numerical calculations) and Glob (for reading the files existing in one paste)

import pandas as pd
import numpy as np
from glob import glob


archives = sorted(glob('../archive/datasets/*.csv')) #give a list of sorted archives with .csv extension in the datasets paste

array = []  #create an array to sort the file's data

for count in archives: #create a function to read and append the file's data
    df_temp= pd.read_csv(count)
    array.append(df_temp)

df = pd.concat(array, axis = 0) #concatenate in one data frame

df.head()

,datahora,satelite,pais,estado,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
0,2010/01/01 15:40:00,AQUA_M-T,Brasil,SERGIPE,JAPOATA,Mata Atlantica,NaN,NaN,NaN,-10.347,-36.777,NaN
1,2010/01/01 15:41:00,AQUA_M-T,Brasil,PERNAMBUCO,PESQUEIRA,Caatinga,NaN,NaN,NaN,-8.442,-36.683,NaN
2,2010/01/01 15:41:00,AQUA_M-T,Brasil,SERGIPE,PORTO DA FOLHA,Caatinga,NaN,NaN,NaN,-9.861,-37.532,NaN
3,2010/01/01 15:41:00,AQUA_M-T,Brasil,PERNAMBUCO,PESQUEIRA,Caatinga,NaN,NaN,NaN,-8.445,-36.667,NaN
4,2010/01/01 17:14:00,AQUA_M-T,Brasil,SANTA CATARINA,SAO JOSE DO CERRITO,Mata Atlantica,NaN,NaN,NaN,-27.542,-50.802,NaN


In [2]:
df.describe()

,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
count,1.332642e+06,1.332642e+06,1.332642e+06,2.160585e+06,2.160585e+06,550668.000000
mean,7.755041e+00,6.945949e-01,-4.435610e+00,-1.013861e+01,-5.145434e+01,61.057242
std,8.675142e+01,2.960866e+00,7.189280e+01,6.252842e+00,7.422538e+00,133.253760
min,-9.990000e+02,0.000000e+00,-9.990000e+02,-3.370900e+01,-7.367400e+01,0.000000
25%,1.000000e+00,0.000000e+00,5.000000e-01,-1.339200e+01,-5.604900e+01,15.200000
50%,5.000000e+00,0.000000e+00,1.000000e+00,-9.345000e+00,-5.040800e+01,28.200000
75%,1.500000e+01,1.000000e-01,1.000000e+00,-5.873000e+00,-4.577000e+01,58.600000
max,1.200000e+02,1.255000e+02,1.000000e+00,5.154000e+00,-3.481600e+01,8589.800000


As we see, we don't see so many informations here, let's see how many nulled values we have

In [3]:
df.isnull().sum()

datahora              0
satelite              0
pais                  0
estado                0
municipio             0
bioma                 0
diasemchuva      827943
precipitacao     827943
riscofogo        827943
latitude              0
longitude             0
frp             1609917
dtype: int64

The diasemchuva, precipitacao, riscofogo and frp columns are the ones with nulled values

## Step two - cleaning the dataset

### Now it's time to preparate the dataset by apply some methods

We need to change some columns names in english translation, to facilitate our understanding

In [4]:
df = df.rename( columns = {'datahora':'datetime', 'satelite':'satellite', 'pais':'country', 'estado':'state', 'municipio':'county', 'bioma':'biome', 'diasemchuva':'daywithoutrain', 'precipitacao':'rainfall', 'riscofogo':'firerisk'})

df.columns

Index(['datetime', 'satellite', 'country', 'state', 'county', 'biome',
       'daywithoutrain', 'rainfall', 'firerisk', 'latitude', 'longitude',
       'frp'],
      dtype='object')

After this, let´s begin with our strategy

#### Filter dataframe and change formats

To filter our dataframe, we have to consider only countys with the "amazon" biome. After the filtering, we change the biome's name for the english translation

In [5]:
df['biome'].unique() # let's see what brazilian's biomes are showed 

array(['Mata Atlantica', 'Caatinga', 'Pampa', 'Cerrado', 'Amazonia',
       'Pantanal'], dtype=object)

In [6]:
# the portuguese translation for Amazon is "Amazonia". Let's filter rows with this biome and change it's name

df = df[df.biome == 'Amazonia'] #filtering rows with "Amazonia" 
df['biome'] = df['biome'].map({'Amazonia':'Amazon'}, na_action=None) #replace velues with the english translation

df.head()

,datetime,satellite,country,state,county,biome,daywithoutrain,rainfall,firerisk,latitude,longitude,frp
15,2010/01/01 17:19:00,AQUA_M-T,Brasil,MATO GROSSO,COLNIZA,Amazon,NaN,NaN,NaN,-9.465,-59.199,NaN
16,2010/01/01 17:21:00,AQUA_M-T,Brasil,AMAZONAS,TEFE,Amazon,NaN,NaN,NaN,-3.466,-64.524,NaN
17,2010/01/01 17:21:00,AQUA_M-T,Brasil,AMAZONAS,TEFE,Amazon,NaN,NaN,NaN,-3.462,-64.492,NaN
18,2010/01/01 17:21:00,AQUA_M-T,Brasil,AMAZONAS,TEFE,Amazon,NaN,NaN,NaN,-3.657,-64.586,NaN
19,2010/01/01 17:21:00,AQUA_M-T,Brasil,AMAZONAS,TEFE,Amazon,NaN,NaN,NaN,-3.649,-64.580,NaN


In [7]:
df.shape #let's see how many observations we have

(983081, 12)

Now, we have to drop some redundant columns (Like country and satellite), because it's only one value

In [8]:
df = df.drop(columns = ['satellite', 'country', 'frp'])

df = df.reset_index().drop(columns = ['index']) #that's something i forget to do in the previous step

df.head()

,datetime,state,county,biome,daywithoutrain,rainfall,firerisk,latitude,longitude
0,2010/01/01 17:19:00,MATO GROSSO,COLNIZA,Amazon,NaN,NaN,NaN,-9.465,-59.199
1,2010/01/01 17:21:00,AMAZONAS,TEFE,Amazon,NaN,NaN,NaN,-3.466,-64.524
2,2010/01/01 17:21:00,AMAZONAS,TEFE,Amazon,NaN,NaN,NaN,-3.462,-64.492
3,2010/01/01 17:21:00,AMAZONAS,TEFE,Amazon,NaN,NaN,NaN,-3.657,-64.586
4,2010/01/01 17:21:00,AMAZONAS,TEFE,Amazon,NaN,NaN,NaN,-3.649,-64.580


Now, we are about to see the column's type of data

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983081 entries, 0 to 983080
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   datetime        983081 non-null  object 
 1   state           983081 non-null  object 
 2   county          983081 non-null  object 
 3   biome           983081 non-null  object 
 4   daywithoutrain  642370 non-null  float64
 5   rainfall        642370 non-null  float64
 6   firerisk        642370 non-null  float64
 7   latitude        983081 non-null  float64
 8   longitude       983081 non-null  float64
dtypes: float64(5), object(4)
memory usage: 67.5+ MB


As we can see, the "datetime" column are labeled as an object type (and doesn't), let's change this for a datetime format

In [10]:
df['datetime'] = pd.to_datetime(df['datetime'], errors = 'coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983081 entries, 0 to 983080
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   datetime        983081 non-null  datetime64[ns]
 1   state           983081 non-null  object        
 2   county          983081 non-null  object        
 3   biome           983081 non-null  object        
 4   daywithoutrain  642370 non-null  float64       
 5   rainfall        642370 non-null  float64       
 6   firerisk        642370 non-null  float64       
 7   latitude        983081 non-null  float64       
 8   longitude       983081 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 67.5+ MB


Now, we have our columns correctly formatted.

#### Working with dates and times

In [11]:
#Importing the datetime library for datetime manipulation

from datetime import datetime

In [12]:
df['date'] = df['datetime'].dt.date

df = df.drop(columns = ['datetime'])

df['date'] = pd.to_datetime(df['date'], errors = 'coerce')

df.head()

,state,county,biome,daywithoutrain,rainfall,firerisk,latitude,longitude,date
0,MATO GROSSO,COLNIZA,Amazon,NaN,NaN,NaN,-9.465,-59.199,2010-01-01
1,AMAZONAS,TEFE,Amazon,NaN,NaN,NaN,-3.466,-64.524,2010-01-01
2,AMAZONAS,TEFE,Amazon,NaN,NaN,NaN,-3.462,-64.492,2010-01-01
3,AMAZONAS,TEFE,Amazon,NaN,NaN,NaN,-3.657,-64.586,2010-01-01
4,AMAZONAS,TEFE,Amazon,NaN,NaN,NaN,-3.649,-64.580,2010-01-01


#### Fill nulled values

To understand how we can fill these nulled values, let's see how many are

In [13]:
df.isnull().sum()

state                  0
county                 0
biome                  0
daywithoutrain    340711
rainfall          340711
firerisk          340711
latitude               0
longitude              0
date                   0
dtype: int64

We can see a great number of nulled values in some columns. Let's focus on the filled values.

In [14]:
df.tail() #let's see the last rows of the dataset

,state,county,biome,daywithoutrain,rainfall,firerisk,latitude,longitude,date
983076,AMAZONAS,HUMAITA,Amazon,2.0,0.0,0.2,-7.967,-63.496,2020-07-10
983077,MATO GROSSO,COLNIZA,Amazon,23.0,0.0,0.7,-9.120,-59.092,2020-07-10
983078,PARA,PARAGOMINAS,Amazon,10.0,0.0,0.3,-3.183,-47.246,2020-07-11
983079,PARA,TOME-ACU,Amazon,10.0,1.8,0.1,-2.840,-48.110,2020-07-11
983080,PARA,PORTEL,Amazon,2.0,1.6,0.1,-2.371,-50.491,2020-07-11


In [15]:
df['state'].unique() #let's see the states that the amazon rainforest covers

array(['MATO GROSSO', 'AMAZONAS', 'RORAIMA', 'RONDONIA', 'MARANHAO',
       'PARA', 'ACRE', 'AMAPA', 'TOCANTINS'], dtype=object)

In "state" column, some states belongs to the north region, but two states (MATO GROSSO and MARANHAO) are not. In a quick research, we can found that MATO GROSSO belongs to middlewest region and MARANHAO belongs to northeast region. To fullfill the dataset with more information, let's include those informations in a column called "region".

After this, we need to classify our risk in low, medium and high risk. To do this, we're do tha same steps of the "region" column

In [16]:
condition = [df['state']=='MARANHAO', df['state']=='MATO GROSSO']
results = ['NORTHEAST', 'MIDDLEWEST']

df['region'] = np.select(condition,results,'NORTH')

df.tail()

,state,county,biome,daywithoutrain,rainfall,firerisk,latitude,longitude,date,region
983076,AMAZONAS,HUMAITA,Amazon,2.0,0.0,0.2,-7.967,-63.496,2020-07-10,NORTH
983077,MATO GROSSO,COLNIZA,Amazon,23.0,0.0,0.7,-9.120,-59.092,2020-07-10,MIDDLEWEST
983078,PARA,PARAGOMINAS,Amazon,10.0,0.0,0.3,-3.183,-47.246,2020-07-11,NORTH
983079,PARA,TOME-ACU,Amazon,10.0,1.8,0.1,-2.840,-48.110,2020-07-11,NORTH
983080,PARA,PORTEL,Amazon,2.0,1.6,0.1,-2.371,-50.491,2020-07-11,NORTH


In [17]:
condition = [df['firerisk']<=0.3, df['firerisk']<=0.7]
results = ['low', 'medium']

df['risk'] = np.select(condition,results,'high')

df.tail()

,state,county,biome,daywithoutrain,rainfall,firerisk,latitude,longitude,date,region,risk
983076,AMAZONAS,HUMAITA,Amazon,2.0,0.0,0.2,-7.967,-63.496,2020-07-10,NORTH,low
983077,MATO GROSSO,COLNIZA,Amazon,23.0,0.0,0.7,-9.120,-59.092,2020-07-10,MIDDLEWEST,medium
983078,PARA,PARAGOMINAS,Amazon,10.0,0.0,0.3,-3.183,-47.246,2020-07-11,NORTH,low
983079,PARA,TOME-ACU,Amazon,10.0,1.8,0.1,-2.840,-48.110,2020-07-11,NORTH,low
983080,PARA,PORTEL,Amazon,2.0,1.6,0.1,-2.371,-50.491,2020-07-11,NORTH,low


#### Strategies for analyze data

Let´s clean our dataset to have the most non nulled values.

In [18]:
df_not_navalues = df.dropna() #dropping na values

df_not_navalues.shape

(642370, 11)

In [19]:
df_not_navalues.isnull().sum()

state             0
county            0
biome             0
daywithoutrain    0
rainfall          0
firerisk          0
latitude          0
longitude         0
date              0
region            0
risk              0
dtype: int64

As we can see, we have no nulled values. Otherwise, our timeline was completely reduced as we see on the next line

In [20]:
df_not_navalues['date'].dt.year.unique() #the years for analyze

array([2014, 2015, 2016, 2017, 2018, 2019, 2020], dtype=int64)

With database cleaned, let's make some analysis

# For now, we're continue our project in tableau!!

In [21]:
df_not_navalues.to_csv('data_cleaned.csv', index=False) #exporting data for a .csv file